In [1]:
# from english_words import english_words_set\
# import multiprocessing as mp
# import warnings
import pandas as pd
# import numpy as np

In [2]:
f = open('guesses.txt','r')
guesses = []

for line in f:
    guesses.append(line.strip())

print(str(len(guesses)))

12972


In [3]:
words5 = guesses

In [4]:
# this should be an empty list, checking to be sure we do not have non alpha chars in the words
print([i for i in words5 if not i.isalpha()])

[]


In [5]:
len(words5)

12972

In [19]:
large_ranks_dataset_no_dup = 'large_rank_no_doubles'
large_ranks_dataset = 'large_word_ranks'
df_word_ranks = pd.read_csv(large_ranks_dataset)
df_word_ranks_no_doubles = pd.read_csv(large_ranks_dataset_no_dup)

In [20]:
df_ranks = df_word_ranks[['guess','score_match','score_exact_match','solution','double']].copy()

In [21]:
df_ranks['total_score'] = (df_ranks['score_match'] + df_ranks['score_exact_match'])

In [22]:
df_ranks.head()

,guess,score_match,score_exact_match,solution,double,total_score
0,aahed,3973.0,118.0,NaN,True,4091.0
1,aalii,4019.0,269.0,NaN,True,4288.0
2,aargh,3557.0,90.0,NaN,True,3647.0
3,aarti,4257.0,140.0,NaN,True,4397.0
4,abaca,3695.0,511.0,NaN,True,4206.0


In [23]:
print (df_ranks['guess'][df_ranks['total_score'].idxmax()])
print (df_ranks['total_score'][df_ranks['total_score'].idxmax()])

resee
5923.0


In [24]:
def remove_notcontains(grey,df):
    df_sub = df
    for l in grey:
        df_sub = (df_sub.loc[(~df_sub['guess'].str.contains(l))])
    return df_sub

In [25]:
def remove_by_contains(letters,df):
    df_sub = df
    for l in letters:
        df_sub = (df_sub.loc[(df_sub['guess'].str.contains(l))])
    return df_sub

In [26]:
def remove_by_location(loc,letter,df):
    df_sub = df
    loc = loc-1 # for easier use 0 is positon 1
    for l in letter:
        df_sub = (df_sub.loc[(df_sub['guess'].str[loc]==l)])
    return df_sub

In [27]:
# confirm letter not in location!
def remove_by_not_location(loc,letter,df):
    df_sub = df
    loc = loc-1 # for easier use 0 is positon 1
    for l in letter:
        df_sub = (df_sub.loc[(df_sub['guess'].str[loc]!=l)])
    return df_sub

In [28]:
def remove_by_letter_count(letter,count,df):
    df_sub = df
    for l in letter:
        df_sub = (df_sub.loc[(df_sub['guess'].str.count(letter)==count)])
    return df_sub

In [29]:
# df_ranks = df_ranks.loc[(df_ranks['solution']==True)]

In [30]:
print(df_ranks.head())
print('strongest start suggestion')
print (df_ranks['guess'][df_ranks['total_score'].idxmax()])
print (df_ranks['total_score'][df_ranks['total_score'].idxmax()])

   guess  score_match  score_exact_match solution double  total_score
0  aahed       3973.0              118.0      NaN   True       4091.0
1  aalii       4019.0              269.0      NaN   True       4288.0
2  aargh       3557.0               90.0      NaN   True       3647.0
3  aarti       4257.0              140.0      NaN   True       4397.0
4  abaca       3695.0              511.0      NaN   True       4206.0
strongest start suggestion
resee
5923.0


here is the game aid loop

In [31]:
# make the testing loop
# starting DF df_word_ranks

# show results and sort non-doubles over doubles after score value

round = 0
word = ''
df_options = df_ranks

print('strongest start suggestion')
print (df_ranks['guess'][df_ranks['total_score'].idxmax()])
print (df_ranks['total_score'][df_ranks['total_score'].idxmax()])

go = 'go'

while go != 'no':
    round += 1 
    if round == 1:

        print('initial guess')
        word = input('enter your guess:')
        pass

    if round > 1:

        print(str(round))

        not_char = input('enter eliminated letters:[l,e,t,t,e,r]')
        is_char = input('enter confirmed letters:[y,e,s]')
        loc_letters = input('enter confirmed locations: [letter,location]')
        dup_letters = input('enter known letter counts: [letter,count]')
        not_loc_letters = input('for known letters, where are they not located? : [letter,count]')

        # eliminate options, and show the reduced best 5 choices
        if len(not_char)>0:
            df_options = remove_notcontains(not_char,df_options)

        if len(is_char)>0:
            df_options = remove_by_contains(is_char,df_options)
            if len(not_loc_letters) >0:
                for i in range(0,int(len(not_loc_letters)),2):
                    df_options = remove_by_not_location(loc=int(not_loc_letters[i+1]),letter=str(not_loc_letters[i]),df=df_options)

        if len(loc_letters)>0:
            # remove by letters in correct location
            for i in range(0,int(len(loc_letters)),2):
                df_options = remove_by_location(loc=int(loc_letters[i+1]),letter=str(loc_letters[i]),df=df_options)
        
        if len(dup_letters)>0:
            # remove by letters in correct location
            for i in range(0,int(len(loc_letters)),2):
                df_options = remove_by_letter_count(loc=int(dup_letters[i+1]),letter=str(dup_letters[i]),df=df_options)
        
        try:
            next_best = (df_options['guess'][df_options['total_score'].idxmax()])
            print('from the results of the last guess')
            print('next best option is ' + str(next_best))
            print('your top remaiing options are: ')
            print(df_options.nlargest(3, 'total_score'))
        except:
            print('message')
            print(df_options.head())
            pass
    
    
        go = input('go again?:')

    print(str(round))


strongest start suggestion
resee
5923.0
initial guess
1
2
from the results of the last guess
next best option is cloot
your top remaiing options are: 
      guess  score_match  score_exact_match solution double  total_score
2111  cloot       3433.0              162.0      NaN   True       3595.0
7743  onion       3329.0              222.0     True   True       3551.0
8291  pitot       3250.0              228.0      NaN   True       3478.0
2


In [ ]:
#  this can be improved by confiming the number of letters in a word, 'wince' has one E and this was confirmed
# use this to elimate more from the pool when the information is availible